In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
credit_card = pd.read_csv('creditcard.csv')
credit_card.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
credit_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
credit_card.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
# Check the distribution of legit transactions & fraudulent transactions
credit_card['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

The result above shows that the data is not balanced, as there is a big difference between class 0 and 1.
* 0 = legit transaction
* 1 = Fraudulent transaction

In [6]:
# separate the data for analysis
legit = credit_card[credit_card.Class == 0]
fraud = credit_card[credit_card.Class == 1]

print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [7]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [8]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [9]:
# compare the values for both transactions
credit_card.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Build a dataset with the same amount of transactions with fraudulent data(492)

In [10]:
legit_data = legit.sample(n=492)

### Concatenate legit data and fraudulent data

In [11]:
new_data = pd.concat([legit_data, fraud], axis=0)
new_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
176353,122728.0,-0.201186,0.446825,1.497623,0.445010,0.370437,1.958762,-0.274738,0.557853,0.137112,...,0.335178,1.035274,-0.216047,-0.758449,-0.441370,-0.422239,0.266655,0.192749,45.70,0
166571,118172.0,-1.242166,0.673920,1.081235,-1.330230,-0.209052,0.781606,0.887542,-1.935680,0.808511,...,1.344250,-0.358906,0.280639,0.464456,-0.143658,-0.895739,0.192444,-0.158415,175.96,0
233304,147519.0,-6.179983,-5.180311,-1.420568,1.671558,0.248411,-1.535615,0.788957,-0.083684,0.355708,...,-0.470347,-0.395939,-0.746648,0.296618,-0.114551,0.047250,0.412185,-1.646735,238.17,0
42616,41173.0,-1.650815,-1.462619,1.884200,-2.307975,0.390937,-0.546978,-0.543434,0.464041,1.340455,...,0.392831,0.723176,0.113729,-0.281800,0.187262,-0.014348,0.027591,0.111704,105.00,0
159861,113009.0,2.119299,-0.143897,-1.761082,0.029529,0.449121,-0.480476,0.052156,-0.154922,0.984516,...,0.211493,0.707355,-0.048619,0.014669,0.385905,-0.402865,-0.002215,-0.056997,1.00,0


In [12]:
new_data['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [13]:
new_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,97535.252033,-0.046498,-0.048351,-0.065323,-0.148329,-0.008463,0.095853,-0.022817,0.016952,-0.121533,...,-0.046856,-0.051176,0.000329,-0.024024,0.010999,-0.008704,0.014668,-0.009879,0.012069,91.445650
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [14]:
x = new_data.drop(columns='Class', axis=1)
y = new_data['Class']

In [15]:
print(x)

            Time        V1        V2        V3        V4        V5        V6  \
176353  122728.0 -0.201186  0.446825  1.497623  0.445010  0.370437  1.958762   
166571  118172.0 -1.242166  0.673920  1.081235 -1.330230 -0.209052  0.781606   
233304  147519.0 -6.179983 -5.180311 -1.420568  1.671558  0.248411 -1.535615   
42616    41173.0 -1.650815 -1.462619  1.884200 -2.307975  0.390937 -0.546978   
159861  113009.0  2.119299 -0.143897 -1.761082  0.029529  0.449121 -0.480476   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [16]:
print(y)

176353    0
166571    0
233304    0
42616     0
159861    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


### Training the Dataset

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

* Note: Above, `Stratify=y` - allows even distribution of 0 & 1 during `training` and `testing`

In [18]:
print(x.shape, x_train.shape, x_test.shape)

(984, 30) (787, 30) (197, 30)


In [19]:
#model = LogisticRegression()
#model.fit(x_train, y_train)

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [21]:
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=200))

# Fit the model
pipeline.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=200))])

In [22]:
pipeline.score(x_test, y_test)

0.9695431472081218